Data preparation for training

In [1]:
import duckdb, os, csv

In [2]:
DB_PATH = "D:/db/meta.duckdb"
con = duckdb.connect(DB_PATH)

con.execute("PRAGMA threads=2;") 
con.execute("SET memory_limit='5GB';") 
con.execute("SET preserve_insertion_order=false;") 
print("Set up ready")

Set up ready


In [3]:
print("Number of rows in the manifest 17-18", con.sql("""select count(*) as tot from images_manifest1718_clean"""))
print("Number of files in the clean image directory", con.sql("""select count(*) as tot from clean_files"""))
print("Number of Null values in the clean image directory", con.sql("""select count(*) as tot from clean_files where filename is null"""))

print("Number of missing files that are in the manifest but not in the directory")
print(con.sql("""
SELECT COUNT(*) AS missing
FROM images_manifest1718_clean im
LEFT JOIN clean_files cf
    ON im.full_image_file = cf.filename
WHERE cf.filename IS NULL;
"""))

Number of rows in the manifest 17-18 ┌─────────┐
│   tot   │
│  int64  │
├─────────┤
│ 8664413 │
└─────────┘

Number of files in the clean image directory ┌─────────┐
│   tot   │
│  int64  │
├─────────┤
│ 8664413 │
└─────────┘

Number of Null values in the clean image directory ┌───────┐
│  tot  │
│ int64 │
├───────┤
│     0 │
└───────┘

Number of missing files that are in the manifest but not in the directory


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┐
│ missing │
│  int64  │
├─────────┤
│       0 │
└─────────┘



In [4]:
# New table filtering only posts with corresponding image in the manifest (one row per post, without adding new rows if the post has more than one image)
# 1 if the post has the image, 0 otherwise
con.execute("""
ALTER TABLE metadata1718_restricted ADD COLUMN has_image BOOLEAN;

UPDATE metadata1718_restricted
SET has_image = 1
FROM images_manifest1718_clean AS im
WHERE im.post_id = metadata1718_restricted.post_id;

UPDATE metadata1718_restricted
SET has_image = 0
WHERE has_image IS NULL;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
print(con.sql("""
SELECT COUNT(*) AS posts_with_img
FROM metadata1718_restricted
WHERE has_image = 1"""))

print(con.sql("""
SELECT COUNT(*) AS posts_without_img
FROM metadata1718_restricted
WHERE has_image = 0"""))

┌────────────────┐
│ posts_with_img │
│     int64      │
├────────────────┤
│        5581301 │
└────────────────┘

┌───────────────────┐
│ posts_without_img │
│       int64       │
├───────────────────┤
│             62038 │
└───────────────────┘



In [6]:
print(con.sql("""SELECT COUNT(*) AS total_rows, COUNT(DISTINCT post_id) as distinct_posts_manifest FROM images_manifest1718_clean """))
print(con.sql("""SELECT COUNT(*) AS total_posts, COUNT(DISTINCT post_id) as distinct_posts_metadata FROM metadata1718_restricted WHERE has_image = 1 """))

┌────────────┬─────────────────────────┐
│ total_rows │ distinct_posts_manifest │
│   int64    │          int64          │
├────────────┼─────────────────────────┤
│    8664413 │                 6861542 │
└────────────┴─────────────────────────┘

┌─────────────┬─────────────────────────┐
│ total_posts │ distinct_posts_metadata │
│    int64    │          int64          │
├─────────────┼─────────────────────────┤
│     5581301 │                 5581301 │
└─────────────┴─────────────────────────┘



In [7]:
con.execute("""CREATE OR REPLACE TABLE metadata1718_ready AS
SELECT * 
FROM metadata1718_restricted
WHERE has_image = 1;

ALTER TABLE metadata1718_ready DROP COLUMN has_image""")

print(con.sql("""
SELECT COUNT(*) AS total_posts
FROM metadata1718_ready
"""))

# Now it coincides with the cleaned manifest

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┐
│ total_posts │
│    int64    │
├─────────────┤
│     5581301 │
└─────────────┘



In [8]:
print(con.sql("""SELECT year, COUNT(*) AS n_posts
FROM metadata1718_ready
GROUP BY year"""))

┌───────┬─────────┐
│ year  │ n_posts │
│ int64 │  int64  │
├───────┼─────────┤
│  2017 │ 1454650 │
│  2018 │ 4126651 │
└───────┴─────────┘



In [9]:
month_year = con.sql("""SELECT month, year, COUNT(*) AS n_posts
FROM metadata1718_ready
GROUP BY month, year
ORDER BY year, month""").fetchdf()
print(month_year)

    month  year  n_posts
0       1  2017    63757
1       2  2017    66436
2       3  2017    81539
3       4  2017    90351
4       5  2017   102548
5       6  2017   110260
6       7  2017   125830
7       8  2017   138774
8       9  2017   143112
9      10  2017   164562
10     11  2017   173070
11     12  2017   194411
12      1  2018   210087
13      2  2018   218667
14      3  2018   269635
15      4  2018   295241
16      5  2018   332222
17      6  2018   346755
18      7  2018   381703
19      8  2018   404207
20      9  2018   398450
21     10  2018   433755
22     11  2018   412325
23     12  2018   423604


# SPLIT

In [10]:
# Add a column which indicates whether the post is in the training, validation or test set
con.execute("""ALTER TABLE metadata1718_ready ADD COLUMN IF NOT EXISTS split VARCHAR;

UPDATE metadata1718_ready
SET split = CASE
WHEN date_day < '2018-11-01' THEN 'train'
WHEN date_day < '2018-12-01' THEN 'validation'
ELSE 'test'
END;""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
# Check percentages of each split over the total posts
print(con.sql("""
-- Percentuali sul totale
WITH c AS (
  SELECT split, COUNT(*) AS n_posts FROM metadata1718_ready GROUP BY split
)
SELECT split, n_posts, ROUND(100.0 * n_posts / SUM(n_posts) OVER (), 2) AS pct
FROM c
ORDER BY split;

"""))

┌────────────┬─────────┬────────┐
│   split    │ n_posts │  pct   │
│  varchar   │  int64  │ double │
├────────────┼─────────┼────────┤
│ test       │  423604 │   7.59 │
│ train      │ 4745372 │  85.02 │
│ validation │  412325 │   7.39 │
└────────────┴─────────┴────────┘



In [12]:
# Check if each category is equally represented in each split

wide = con.sql("""
WITH stats AS (
  SELECT split, category, COUNT(*) AS n_posts
  FROM metadata1718_ready
  GROUP BY split, category
),
tot AS (
  SELECT category, SUM(n_posts) AS tot_posts
  FROM stats GROUP BY category
)
SELECT
  s.category,
  ROUND(100.0 * SUM(CASE WHEN split='train' THEN n_posts ELSE 0 END)/t.tot_posts, 2) AS train_pct_posts,
  ROUND(100.0 * SUM(CASE WHEN split='validation'   THEN n_posts ELSE 0 END)/t.tot_posts, 2) AS val_pct_posts,
  ROUND(100.0 * SUM(CASE WHEN split='test'  THEN n_posts ELSE 0 END)/t.tot_posts, 2) AS test_pct_posts
FROM stats s
JOIN tot t USING (category)
GROUP BY s.category, t.tot_posts
ORDER BY s.category
""").fetchdf()

print(wide)


   category  train_pct_posts  val_pct_posts  test_pct_posts
0    beauty            87.85           6.02            6.12
1    family            85.02           7.30            7.68
2   fashion            84.38           7.77            7.84
3   fitness            86.87           6.61            6.52
4      food            84.10           7.88            8.02
5  interior            83.58           8.11            8.31
6     other            83.25           8.03            8.72
7       pet            86.20           6.74            7.06
8    travel            88.42           5.80            5.78


In [13]:
con.sql("""PRAGMA table_info(metadata1718_ready)""").df()

,cid,name,type,notnull,dflt_value,pk
0,0,filename,VARCHAR,False,None,False
1,1,username,VARCHAR,False,None,False
2,2,like_count,INTEGER,False,None,False
3,3,comment_count,INTEGER,False,None,False
4,4,width,INTEGER,False,None,False
5,5,height,INTEGER,False,None,False
6,6,time_utc,TIMESTAMP,False,None,False
7,7,caption,VARCHAR,False,None,False
8,8,aspect_ratio,DOUBLE,False,None,False
9,9,area,INTEGER,False,None,False


In [14]:
# Check if each ER bins is equally represented in each split

wide = con.sql("""
WITH stats AS (
  SELECT split, er_bins, COUNT(*) AS n_posts
  FROM metadata1718_ready
  GROUP BY split, er_bins
),
tot AS (
  SELECT er_bins, SUM(n_posts) AS tot_posts
  FROM stats GROUP BY er_bins
)
SELECT
  s.er_bins,
  ROUND(100.0 * SUM(CASE WHEN split='train' THEN n_posts ELSE 0 END)/t.tot_posts, 2) AS train_pct_posts,
  ROUND(100.0 * SUM(CASE WHEN split='validation'   THEN n_posts ELSE 0 END)/t.tot_posts, 2) AS val_pct_posts,
  ROUND(100.0 * SUM(CASE WHEN split='test'  THEN n_posts ELSE 0 END)/t.tot_posts, 2) AS test_pct_posts
FROM stats s
JOIN tot t USING (er_bins)
GROUP BY s.er_bins, t.tot_posts
ORDER BY s.er_bins
""").fetchdf()

print(wide)


     er_bins  train_pct_posts  val_pct_posts  test_pct_posts
0       high            86.21           6.89            6.90
1        low            85.43           7.18            7.39
2     medium            85.90           7.02            7.09
3  very_high            84.51           7.67            7.82
4   very_low            82.71           8.32            8.97


In [15]:
con.sql("""SELECT er_bins, split, COUNT(*) AS tot
FROM metadata1718_ready
GROUP BY er_bins, split
ORDER BY split
""").df()

,er_bins,split,tot
0,low,test,84147
1,very_low,test,88749
2,very_high,test,87897
3,medium,test,82505
4,high,test,80306
5,high,train,1004030
6,medium,train,999965
7,very_high,train,949815
8,low,train,972970
9,very_low,train,818592


In [16]:
con.sql("""SELECT category, split, COUNT(*) AS tot
FROM metadata1718_ready
GROUP BY category, split
ORDER BY split
""").df()

,category,split,tot
0,pet,test,8053
1,other,test,73174
2,travel,test,40769
3,beauty,test,15433
4,fashion,test,153459
5,food,test,49292
6,family,test,52751
7,fitness,test,13077
8,interior,test,17596
9,food,train,516905


# UNDERSAMPLING

In [17]:
# Count number of users per category in the training set
users_per_cat = con.sql("""
SELECT category, COUNT(DISTINCT username) AS n_users, COUNT(*) as n_posts
FROM metadata1718_ready
WHERE split='train'
GROUP BY category
ORDER BY n_users
""").fetchdf()
print(users_per_cat)

# The minimum number of users per category must be 544

   category  n_users  n_posts
0       pet      544    98295
1   fitness     1065   174321
2  interior     1116   176985
3    beauty     1448   221423
4      food     3246   516905
5    family     3790   584066
6    travel     4028   623887
7     other     4793   698694
8   fashion    11250  1650796


In [18]:
# Count number of users per category in the training set
users_per_cat = con.sql("""
SELECT er_bins, COUNT(DISTINCT username) AS n_users, COUNT(*) as n_posts
FROM metadata1718_ready
WHERE split='train'
GROUP BY er_bins
ORDER BY n_users
""").fetchdf()
print(users_per_cat)

     er_bins  n_users  n_posts
0   very_low    18590   818592
1  very_high    22801   949815
2        low    24876   972970
3       high    26894  1004030
4     medium    27331   999965


In [19]:
# Define a new table with the names of the users selected for the undersampling
con.execute("""CREATE OR REPLACE TABLE train_users_balanced AS
WITH train_users AS (
  SELECT username, category
  FROM metadata1718_ready
  WHERE split = 'train'
  GROUP BY username, category
),
counts AS (
  SELECT category, COUNT(*) AS n_users
  FROM train_users
  GROUP BY category
),
target AS (
  SELECT MIN(n_users) AS k_min FROM counts
),
ranked AS (
  SELECT
    u.username,
    u.category,
    ROW_NUMBER() OVER (
        PARTITION BY u.category
        ORDER BY hash(u.username)
    ) AS rk
  FROM train_users u
)
SELECT r.username, r.category
FROM ranked r
WHERE r.rk <= (SELECT k_min FROM target);
""")

In [20]:
# Add a column in the original table to flag that the user's posts are in the balanced set
con.execute("""
-- 2) Flagga i post che rimangono nel TRAIN bilanciato
ALTER TABLE metadata1718_ready ADD COLUMN IF NOT EXISTS in_train_balanced BOOLEAN;
UPDATE metadata1718_ready m
SET in_train_balanced = EXISTS (
  SELECT 1 FROM train_users_balanced t WHERE t.username = m.username
)
WHERE m.split = 'train';
""")

# Create a table with only the balanced posts in the training set
con.execute("""
CREATE OR REPLACE TABLE train_balanced AS
SELECT * FROM metadata1718_ready WHERE split='train' AND in_train_balanced;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [21]:
con.sql("""
SELECT COUNT(*) AS training_posts FROM train_balanced
""").df()

,training_posts
0,773497


In [22]:
con.sql("""SELECT split, COUNT(*) FROM metadata1718_ready WHERE split = 'validation' OR split = 'test' GROUP BY split""").df()

,split,count_star()
0,validation,412325
1,test,423604


In [35]:
# Number of users per category in the balanced dataset
print(con.sql("""
SELECT m.category, COUNT(DISTINCT username) AS n_users, COUNT(*) AS n_posts
FROM train_users_balanced b
JOIN metadata1718_ready m USING(username)
WHERE m.split = 'train'
GROUP BY m.category
"""))

┌──────────┬─────────┬─────────┐
│ category │ n_users │ n_posts │
│ varchar  │  int64  │  int64  │
├──────────┼─────────┼─────────┤
│ other    │     544 │   80845 │
│ fashion  │     544 │   76763 │
│ beauty   │     544 │   84553 │
│ family   │     544 │   86468 │
│ pet      │     544 │   98295 │
│ fitness  │     544 │   90206 │
│ interior │     544 │   85693 │
│ travel   │     544 │   83873 │
│ food     │     544 │   86801 │
└──────────┴─────────┴─────────┘



In [36]:
# Number of users per category in the balanced dataset
print(con.sql("""
SELECT er_bins, COUNT(DISTINCT username) AS n_users, COUNT(*) AS n_posts
FROM train_users_balanced b
JOIN metadata1718_ready m USING(username)
WHERE m.split = 'train'
GROUP BY er_bins
"""))

┌───────────┬─────────┬─────────┐
│  er_bins  │ n_users │ n_posts │
│  varchar  │  int64  │  int64  │
├───────────┼─────────┼─────────┤
│ high      │    4220 │  166086 │
│ very_high │    3551 │  135441 │
│ low       │    4031 │  166345 │
│ medium    │    4359 │  166993 │
│ very_low  │    3042 │  138632 │
└───────────┴─────────┴─────────┘



In [14]:
print(con.sql("""
SELECT
  er_bins2,
  COUNT(DISTINCT username) AS n_users,
  COUNT(*) AS n_posts,
  ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS pct_posts
FROM train_users_balanced b
JOIN md1718 m USING(username)
WHERE m.split = 'test'
GROUP BY er_bins2
ORDER BY er_bins2
"""))


┌──────────┬─────────┬─────────┬───────────┐
│ er_bins2 │ n_users │ n_posts │ pct_posts │
│ varchar  │  int64  │  int64  │  double   │
├──────────┼─────────┼─────────┼───────────┤
│ high     │    3509 │   31118 │     50.13 │
│ low      │    2945 │   30957 │     49.87 │
└──────────┴─────────┴─────────┴───────────┘



In [15]:
con.sql("""SELECT COUNT(*) AS empty_captions 
FROM md1718
WHERE caption IS NULL""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,empty_captions
0,13672


In [37]:
# Number of users per category in the balanced dataset
print(con.sql("""
SELECT er_bins, split, COUNT(DISTINCT username) AS n_users, COUNT(*) AS n_posts
FROM metadata1718_ready
WHERE split = 'validation' OR split = 'test'
GROUP BY er_bins, split
ORDER BY split
"""))

┌───────────┬────────────┬─────────┬─────────┐
│  er_bins  │   split    │ n_users │ n_posts │
│  varchar  │  varchar   │  int64  │  int64  │
├───────────┼────────────┼─────────┼─────────┤
│ very_high │ test       │   14185 │   87897 │
│ very_low  │ test       │    9850 │   88749 │
│ low       │ test       │   14516 │   84147 │
│ high      │ test       │   16475 │   80306 │
│ medium    │ test       │   16440 │   82505 │
│ high      │ validation │   16564 │   80291 │
│ medium    │ validation │   16467 │   81667 │
│ very_high │ validation │   13971 │   86194 │
│ very_low  │ validation │    9555 │   82346 │
│ low       │ validation │   14376 │   81827 │
├───────────┴────────────┴─────────┴─────────┤
│ 10 rows                          4 columns │
└────────────────────────────────────────────┘



# DEFINE SET OF IMAGES TO TRAIN

In [38]:
print(con.sql("""
SELECT post_id FROM train_balanced LIMIT 10"""))

print(con.sql("""
SELECT * FROM images_manifest1718_clean LIMIT 10"""))

print(con.sql("""
SELECT * FROM clean_files LIMIT 10"""))

┌───────────────────────────────────┐
│              post_id              │
│              varchar              │
├───────────────────────────────────┤
│ jessieonair-1832772436464878417   │
│ jess_soothill-1814050234055839189 │
│ jess_soothill-1815434266849071530 │
│ jess_soothill-1816220883029348816 │
│ jess_soothill-1817582292300511649 │
│ jess_soothill-1819123349978893787 │
│ jess_soothill-1824907412828563208 │
│ jess_soothill-1827800193871699807 │
│ jess_soothill-1828550938351069021 │
│ jess_soothill-1831438017313427228 │
├───────────────────────────────────┤
│              10 rows              │
└───────────────────────────────────┘

┌───────────────┬─────────────────────┬─────────────────────────┬───────────────────────────────────┬───────────────────────────────────────┬───────┐
│   username    │        post         │       image_file        │              post_id              │            full_image_file            │  rn   │
│    varchar    │       varchar       │         varch

In [39]:
con.execute("""
CREATE OR REPLACE VIEW images_train AS
WITH img_filenames AS (
    SELECT 
        imm.full_image_file,
        imm.post_id
    FROM images_manifest1718_clean AS imm
    JOIN train_balanced AS tr
        ON imm.post_id = tr.post_id
)
SELECT 
    imgf.full_image_file,
    imgf.post_id
FROM img_filenames AS imgf
JOIN clean_files AS cl
    ON imgf.full_image_file = cl.filename;
    """)

In [44]:
con.close()